In [ ]:
import tensorflow as tf
print(tf.__version__)

The Fashion MNIST data is available directly in the tf.keras datasets API

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

Images to train with the corresponding labels

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

We notice that all of the values in the number are between 0 and 255. If we are training a neural network, for various reasons it's easier if we treat all values as between 0 and 1, a process called '**normalizing**'.

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

Create a callback funditon which we will need later in the notebook. The function is used to cancel the training once an accuracy of 90% is reached.


In [ ]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

Create a sequential model object:

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

**Sequential**: That defines a **sequence** of layers in the neural network

**Flatten** Flatten takes that square and turns it into a 1 dimensional set/array.

**Dense**: Adds a layer of neurons. Each layer of neurons need an **activation function** to tell them what to do. 

**Relu** effectively means "If X>0 return X, else return 0" -- so what it does it it only passes values 0 or greater to the next layer in the network.

**Softmax** takes a set of values, and effectively picks the biggest one. If the output of the last layer looks like [0.2, 0.5, 0.01, 0.21, 6.5, 0.12, 0.2, 0.1, 0.34], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0]

Now we train the model...

In [ ]:
callback = Callback()
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=3, callbacks = [callback])

...and evaluate it.

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

The training accuracy is 88 %, which is not bad but also not great. For the test set the accuracy is 86 %, which says that our model is not overfitting the train data to well. Also consider that the model was trained only for 3 epochs. For a better resutl one can incresea the numner

# MLFLOW

The MLFLOW URI should not be stored on git. Save it locally in the .mlflow_uri file:

In [ ]:
!echo http://127.0.0.1:5000/ > .mlflow_uri

In [ ]:
!mlflow experiments create --experiment-name 0-ds-computer-vision

In [ ]:
!mlflow ui